In [1]:
from os import getcwd, chdir
import re
import numpy as np
import pickle as pk
import pandas as pd

from nltk.metrics import ConfusionMatrix
from nltk.classify import NaiveBayesClassifier, MaxentClassifier
from nltk.classify import accuracy
from nltk.tokenize import word_tokenize as wt

from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import chi2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score

In [10]:
# open the files for negative and positive sentiment mining
ffile1 = open("D:\\EBAC\\Semester 2\\New Media and Sentiment Mining\\Assignment\\Trainset-pos.csv","r", encoding = 'utf-8',errors='ignore')
ffile2 = open("D:\\EBAC\\Semester 2\\New Media and Sentiment Mining\\Assignment\\Trainset-neg.csv","r", encoding = 'utf-8',errors='ignore')

df_pos = pd.read_csv(ffile1)
df_neg = pd.read_csv(ffile2)
ffile1.close()
ffile2.close()

print (df_pos.head(3))
print (df_neg.head(3))

print(len(df_pos))
print(len(df_neg))

     Type               Restaurant_Name  \
0  Indian  Murugan Idli Shop, Singapore   
1  Indian  Murugan Idli Shop, Singapore   
2  Indian  Murugan Idli Shop, Singapore   

                                        Review_Title  Sentiment  Rating  \
0                                  Soft hot sponges!          1      40   
1                                     Excellent Idli          1      50   
2  perfect for a weekend breakfast- if you love i...          1      40   

  Review_Date                                     Review_Content  
0   26-Oct-10  Home made idlies to the best! With varieties o...  
1   17-Aug-10  I have never had such a soft idli's in my life...  
2   14-Apr-10  one of the best places in Singapore for idlis ...  
     Type                     Restaurant_Name  \
0  Indian  Muthu's Curry @ Dempsey, Singapore   
1  Indian  Muthu's Curry @ Dempsey, Singapore   
2  Indian  Muthu's Curry @ Dempsey, Singapore   

                             Review_Title  Sentiment  Rating 

In [11]:
# TRAINING SET
train_pos = [[x, 1] for x in df_pos["Review_Content"]]
train_neg = [[x, -1] for x in df_neg["Review_Content"]]

trainset = train_pos + train_neg
train_tokenized = [[wt(x), c] for x,c in trainset]

def word_feats(words):
    return dict([(word, True) for word in words])
train_featureset = [(word_feats(d), c) for (d,c) in train_tokenized] 

In [12]:
# STANDARD SET
ffile1 = open("D:\\EBAC\\Semester 2\\New Media and Sentiment Mining\\Assignment\\Train_Standardset-sentiment.csv","r", encoding = "ISO-8859-1",errors='ignore')
df = pd.read_csv(ffile1, encoding = "utf-8")

train_pos = [[row["Review_Content"], 1] for idx, row in df.iterrows() if row["Sentiment"]=="positive"]
train_neg = [[row["Review_Content"], -1] for idx, row in df.iterrows() if row["Sentiment"]=="negative"]

trainset_std = train_pos + train_neg
train_tokenized_std = [[wt(x), c] for x,c in trainset_std]

def word_feats(words):
    return dict([(word, True) for word in words])
train_featureset_std = [(word_feats(d), c) for (d,c) in train_tokenized_std] 

In [13]:
## Naive Bayes Rule using nltk for TRAINING SET
classifier_nb = NaiveBayesClassifier.train(train_featureset)
#print("Accuracy :" +str(accuracy(classifier_nb, test_featureset)))
classifier_nb.show_most_informative_features(10)  # -10 to see positive

## Preparing the data first 
train_nolab = [t[0] for t in trainset]
train_lab = [t[1] for t in trainset]

# Create your tf-idf function
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(train_nolab)

## train Naive Bayes Rule using sklearn
clf = MultinomialNB().fit(train_vectors, train_lab)

predNB = clf.predict(train_vectors)
pred = list(predNB)
cm1=pd.crosstab( pd.Series(train_lab), pd.Series(pred), rownames= ['actuals'], colnames=['pred'],margins=True)
cm1

Most Informative Features
                   worse = True               -1 : 1      =     26.6 : 1.0
                     rip = True               -1 : 1      =     26.0 : 1.0
                   worst = True               -1 : 1      =     25.0 : 1.0
                    rude = True               -1 : 1      =     22.6 : 1.0
                mediocre = True               -1 : 1      =     20.7 : 1.0
                horrible = True               -1 : 1      =     19.8 : 1.0
                 refused = True               -1 : 1      =     19.0 : 1.0
                    poor = True               -1 : 1      =     18.1 : 1.0
                terrible = True               -1 : 1      =     16.2 : 1.0
                 ignored = True               -1 : 1      =     16.2 : 1.0


pred,-1,1,All
actuals,,,
-1,310,1076,1386
1,2,2921,2923
All,312,3997,4309


In [14]:
## Naive Bayes Rule using nltk for STANDARD SET
classifier_nb_std = NaiveBayesClassifier.train(train_featureset_std)
#print("Accuracy :" +str(accuracy(classifier_nb, test_featureset)))
classifier_nb_std.show_most_informative_features(10)  

## Preparing the data first 
train_nolab_std = [t[0] for t in trainset_std]
train_lab_std = [t[1] for t in trainset_std]

# Create your tf-idf function
vectorizer_std = TfidfVectorizer()
train_vectors_std = vectorizer_std.fit_transform(train_nolab_std)

## train Naive Bayes Rule using sklearn
clf_std = MultinomialNB().fit(train_vectors_std, train_lab_std)

predNB_std = clf_std.predict(train_vectors_std)
pred_std = list(predNB_std)
cm2=pd.crosstab( pd.Series(train_lab_std), pd.Series(pred_std), rownames= ['actuals'], colnames=['pred'],margins=True)
cm2

Most Informative Features
                   worst = True               -1 : 1      =     38.3 : 1.0
                   worse = True               -1 : 1      =     35.0 : 1.0
               Excellent = True                1 : -1     =     25.4 : 1.0
                    rude = True               -1 : 1      =     25.0 : 1.0
                terrible = True               -1 : 1      =     20.9 : 1.0
                mediocre = True               -1 : 1      =     19.3 : 1.0
                    Ngee = True               -1 : 1      =     17.1 : 1.0
                   awful = True               -1 : 1      =     17.1 : 1.0
                    poor = True               -1 : 1      =     14.6 : 1.0
                horrible = True               -1 : 1      =     14.4 : 1.0


pred,-1,1,All
actuals,,,
-1,249,1160,1409
1,2,2898,2900
All,251,4058,4309


In [19]:
## Testing the model
## Preparing the data first as per usual
ffile1 = open("D:\\EBAC\\Semester 2\\New Media and Sentiment Mining\\Assignment\\Testset-pos.csv","r", encoding = 'utf-8',errors='ignore')
ffile2 = open("D:\\EBAC\\Semester 2\\New Media and Sentiment Mining\\Assignment\\Testset-neg.csv","r", encoding = 'utf-8',errors='ignore')
df_pos = pd.read_csv(ffile1)
df_neg = pd.read_csv(ffile2)
ffile1.close()
ffile2.close()
test_pos = [[x, 1] for x in df_pos["Review_Content"]]
test_neg = [[x, -1] for x in df_neg["Review_Content"]]
testset = test_pos + test_neg
test_nolab = [t[0] for t in testset]
test_lab = [t[1] for t in testset]
test_tokenized = [[wt(x), c] for x,c in testset]
test_featureset = [(word_feats(d), c) for (d,c) in test_tokenized] 
test_nolab_tok = [t[0] for t in test_featureset]  # need to transform to predict

test_vectors = vectorizer.transform(test_nolab)  
test_vectors_std = vectorizer_std.transform(test_nolab)

In [ ]:
#test_nolab

In [20]:
# using split dataset
predNB = clf.predict(test_vectors)
predictions_nb = []
for t in test_nolab_tok:
    predictions_nb.append(classifier_nb.classify(t))
cm3=pd.crosstab( pd.Series(test_lab), pd.Series(predNB), rownames= ['actuals'], colnames=['pred'],margins=True)
print (cm3)
Accuracy = (cm3[1][1]+cm3[-1][-1])/(cm3[1][1]+(cm3[1][-1])+cm3[-1][1]+(cm3[-1][-1]))
print ("Accuracy is " + str(round(Accuracy,3)))

pred     -1     1   All
actuals                
-1       43   556   599
1         2  1251  1253
All      45  1807  1852
Accuracy is 0.699


In [22]:
# this is for the std Naive Bayes model using a larger and standard data set
predNB_std = clf_std.predict(test_vectors_std)
predictions_nb_std = []
for t in test_nolab_tok:
    predictions_nb_std.append(classifier_nb_std.classify(t))
cm4=pd.crosstab( pd.Series(test_lab), pd.Series(predNB_std), rownames= ['actuals'], colnames=['pred'],margins=True)
print (cm4)
Accuracy = (cm4[1][1]+cm4[-1][-1])/(cm4[1][1]+(cm4[1][-1])+cm4[-1][1]+(cm4[-1][-1]))
print ("Accuracy is " + str(round(Accuracy,3)))

pred     -1     1   All
actuals                
-1       72   527   599
1         0  1253  1253
All      72  1780  1852
Accuracy is 0.715


In [23]:
#Maximum Entropy Classifier (MaxEnt)
import time
stime = time.time()
#classifier_me = MaxentClassifier.train(train_featureset_std, algorithm="IIS", max_iter=5)
# takes like 30 mins for 5 iterations
print("--- %s seconds ---" % (time.time() - stime))
# IIS - improved iterative scaling - optimisation algorithm
#pk.dump(classifier_me, open(fpath+"\\Data\\maxent_std.pk","wb"))
#pk.dump(classifier_me, open(fpath+"\\Data\\maxent.pk","wb"))
# pickle the classifier
classifier_me = pk.load(open("D:\\EBAC\\Semester 2\\New Media and Sentiment Mining\\Assignment\\maxent_std.pk","rb"))
#classifier_me = pk.load(open(fpath+"\\Data\\maxent.pk","rb"))

print(accuracy(classifier_me, test_featureset))

wts = classifier_me.weights()
print (len(wts))

--- 0.0 seconds ---
0.6765658747300216
74085


In [24]:
pred_me = []
classifier_me = pk.load(open("D:\\EBAC\\Semester 2\\New Media and Sentiment Mining\\Assignment\\maxent.pk","rb"))
for t in test_nolab_tok:
    pred_me.append(classifier_me.classify(t))

cm5=pd.crosstab( pd.Series(test_lab), pd.Series(pred_me), rownames= ['actuals'], colnames=['pred'],margins=True)
print (cm5)

pred     -1     1   All
actuals                
-1        1   598   599
1         0  1253  1253
All       1  1851  1852


In [25]:
# Negative tagging
def neg_tag(text):
    transformed = re.sub(r"\b(?:never|nothing|nowhere|noone|none|not|haven't|hasn't|hasnt|hadn't|hadnt|can't|cant|couldn't|couldnt|shouldn't|shouldnt|won't|wont|wouldn't|wouldnt|don't|dont|doesn't|doesnt|didn't|didnt|isnt|isn't|aren't|arent|aint|ain't|hardly|seldom)\b[\w\s]+[^\w\s]", lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG_\2', match.group(0)), text, flags=re.IGNORECASE)
    return(transformed)

text = "I don't like that place you keep calling awesome."
print (neg_tag(text))

# Create a training list which will now contain reviews with Negatively tagged words and their labels
train_set_neg = []

# Append elements to the list
for doc in trainset:
    trans = neg_tag(doc[0])
    lab = doc[1]
    train_set_neg.append([trans, lab])

# Create a testing list which will now contain reviews with Negatively tagged words and their labels
test_set_neg = []

# Append elements to the list
for doc in testset:
    trans = neg_tag(doc[0])
    lab = doc[1]
    test_set_neg.append([trans, lab])

I don't NEG_like NEG_that NEG_place NEG_you NEG_keep NEG_calling NEG_awesome.


In [26]:
# Using tf-idf as features for training
# sklearn library
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

vectorizer = TfidfVectorizer()  # tf-idf compared to term-document matrix

# this is used below for training the SVM
train_vectors = vectorizer.fit_transform(train_nolab)   # uses the UGC data set to classify the downloaded texts from Yelp
test_vectors = vectorizer.transform(test_nolab)  
#pk.dump(vectorizer, open(fpath+"\\Data\\vectorise.pk","wb"))

# Ascertain shape of the sparse matrix
print ("Train matrix shape " + str(train_vectors.shape))
print ("Test matrix shape " + str(test_vectors.shape))


Train matrix shape (4309, 11779)
Test matrix shape (1852, 11779)


In [27]:
print ("Word Features ")
print (vectorizer.get_feature_names()[500:550])
termperdoc = vectorizer.inverse_transform(vectorizer.get_feature_names()[5000:5050])

Word Features 
['admin', 'admiration', 'admiring', 'admit', 'admitt', 'admitted', 'admittedly', 'adore', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'adventurous', 'advertise', 'advertised', 'advertisement', 'advertises', 'advertising', 'advertisings', 'advice', 'adviced', 'advices', 'advisable', 'advise', 'advised', 'adviser', 'advises', 'advising', 'advisor', 'advocates', 'afar', 'affair', 'affected', 'affectionados', 'affiliated', 'affirmative', 'affirmed', 'afford', 'affordable', 'aficionados', 'afield', 'afraid', 'africain', 'aft', 'after', 'afterall', 'afternoon', 'afternoonlocation', 'aftertaste']


In [28]:
# SVM Classifier from sklearn
def train_svm(X, y):
    """
    Create and train the Support Vector Machine.
    """
    svm = SVC(C=10000.0, gamma='auto', kernel='rbf')
    svm.fit(X, y)
    return svm

# Pickled model as it takes ahwile for generation
classifier_svm = train_svm(train_vectors, train_lab)  # training the SVM model
pk.dump(classifier_svm, open("classifier_svm.pk","wb"))
#classifier_svm = pk.load(open("classifier_svm.pk", "rb"))
predSVM = classifier_svm.predict(test_vectors) 
pred_svm = list(predSVM)

cm6=pd.crosstab( pd.Series(test_lab), pd.Series(pred_svm), rownames= ['actuals'], colnames=['pred'],margins=True)
print (cm6)
print (classification_report(pred_svm,  test_lab))

pred      -1     1   All
actuals                 
-1       385   214   599
1        192  1061  1253
All      577  1275  1852
             precision    recall  f1-score   support

         -1       0.64      0.67      0.65       577
          1       0.85      0.83      0.84      1275

avg / total       0.78      0.78      0.78      1852



In [29]:
# k-nn classifier

def train_knn(X, y, k, weight):
    """
    Create and train the k-nearest neighbor.
    """
    knn = KNeighborsClassifier(n_neighbors = k, weights = weight, metric = 'cosine', algorithm = 'brute')
    knn.fit(X, y)
    return knn

kn = train_knn(train_vectors, train_lab, 3, 'distance')# k=20; distance weights - by inverse of distance
predKN = kn.predict(test_vectors)
pred = list(predKN)
cm = ConfusionMatrix(pred, test_lab)
print(cm)

   |   -     |
   |   1   1 |
---+---------+
-1 |<288>306 |
 1 | 311<947>|
---+---------+
(row = reference; col = test)



In [30]:
# sklearn
def train_dtc(X, y):
    """
    Create and train the Decision Tree Classifier.
    """
    dtc = DecisionTreeClassifier()
    dtc.fit(X, y)
    return dtc

dt = train_dtc(train_vectors, train_lab)
predDT = dt.predict(test_vectors)
pred = list(predDT)
cm = ConfusionMatrix(pred, test_lab)
print(cm)

   |    -      |
   |    1    1 |
---+-----------+
-1 | <303> 231 |
 1 |  296<1022>|
---+-----------+
(row = reference; col = test)



In [31]:
ch21 = SelectKBest(chi2, k=5000)
# Transform your training and testing datasets accordingly
train_Kbest = ch21.fit_transform(train_vectors, train_lab)
test_Kbest = ch21.transform(test_vectors)

# Train your SVM with the k best selected features
sv = train_svm(train_Kbest, train_lab)
predSVM= sv.predict(test_Kbest)
pred = list(predSVM)
cm8 = confusion_matrix(pred, test_lab)
print (cm8)
print (accuracy_score(pred, test_lab))
print (classification_report(pred,  test_lab))

[[ 347  125]
 [ 252 1128]]
0.7964362850971922
             precision    recall  f1-score   support

         -1       0.58      0.74      0.65       472
          1       0.90      0.82      0.86      1380

avg / total       0.82      0.80      0.80      1852



In [33]:
clf = MultinomialNB().fit(train_Kbest, train_lab)
predNB = clf.predict(test_Kbest)
pred = list(predNB)
cm9 = confusion_matrix(pred, test_lab)
print (cm9)
print (accuracy_score(pred, test_lab))
print (classification_report(pred,  test_lab))

# View the selected features
selected_features = list(np.array(vectorizer.get_feature_names())[ch21.get_support()])
#print (selected_features)

[[  85    6]
 [ 514 1247]]
0.7192224622030238
             precision    recall  f1-score   support

         -1       0.14      0.93      0.25        91
          1       1.00      0.71      0.83      1761

avg / total       0.95      0.72      0.80      1852

